# Classification of 1000-class ImageNet by VGG
http://aidiary.hatenablog.com/entry/20180212/1518404395

In [1]:
%load_ext autoreload
%autoreload 2

import os, sys, itertools
from pathlib import Path
sys.path.append('..')

import torch
from torch import nn, optim
from torch.nn import functional as F

#from PreTrainedFIM.model import VGG16_FIM, VGG16bn_FIM
from PreTrainedFIM.repara_model import VGG16bn_FIM
from PreTrainedFIM.util import CIFAR10Worker

In [2]:
pretrained, weight_fixing = True, False

name0 = '_p1' if pretrained else '_p0'
name1 = '_w1' if weight_fixing else '_w0'

params = {'epoch_num': 10,
          'log_interval': 1250}
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

vgg_model = VGG16bn_FIM(pretrained=pretrained, weight_fixing=weight_fixing).to(device)
vgg_model = vgg_model.to_device_child_tensors(device)
vgg_model.not_eval_FIM()

#if torch.cuda.device_count() > 1: 
#    vgg_model = nn.DataParallel(vgg_model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg_model.parameters(), lr=0.0001)
#optimizer = optim.SGD(vgg_model.parameters(), lr=0.01, momentum=0.8, weight_decay=5e-4)
worker = CIFAR10Worker(device, vgg_model, criterion, params)
worker = worker.load_data_loader()
worker.test()

for epoch in range(1, 1 + 1):
    filename = f'{epoch}epoch_VGG16bn' + name0 + name1 + '_jnb.pth'
    worker.train(epoch, optimizer)
    worker = worker.set_save_path(filename)
    worker = worker.save_chckpt()

Files already downloaded and verified
Files already downloaded and verified
Test-set loss: 0.5755
Accuracy: 11.120%
Train Epoch: 1 [0/50000 (0%)]	Loss: 0.294944
Train Epoch: 1 [10000/50000 (20%)]	Loss: 0.183764
Train Epoch: 1 [20000/50000 (40%)]	Loss: 0.022886
Train Epoch: 1 [30000/50000 (60%)]	Loss: 0.143737
Train Epoch: 1 [40000/50000 (80%)]	Loss: 0.182944
Accuracy: 71.132%
====> Epoch: 1 Average loss: 0.1128
